In [4]:
!pip install Sastrawi

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 996.8 kB/s eta 0:00:000:0100:01
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import pandas as pd
import re
from tqdm import tqdm
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
import pandas as pd

# Load the CSV dataset into a Pandas DataFrame
df = pd.read_csv("delapan juni.csv")

# Filter the DataFrame to include only rows where the ticket status is "Selesai"
# and the department is one of the specific three departments
specific_departments = [
"DINAS PEKERJAAN UMUM BINA MARGA CIPTA KARYA",
##"Kepolisian Daerah Jawa Tengah",
"DINAS ENERGI DAN SUMBER DAYA MINERAL",
"DINAS PEMBERDAYAAN MASYARAKAT, DESA, KEPENDUDUKAN DAN PENCATATAN SIPIL",
#"Kabupaten Demak",
#"Kabupaten Grobogan",
"DINAS KESEHATAN",
"DINAS TENAGA KERJA DAN TRANSMIGRASI",
#"Kabupaten Banyumas",
#"Kabupaten Blora",
#"Kabupaten Brebes",
"BADAN PENGELOLA PENDAPATAN DAERAH",
#"Kabupaten Kendal",
#"Kabupaten Klaten",
#"Kabupaten Semarang",
#"Kabupaten Pati",
#"Kabupaten Jepara",
#"Kabupaten Boyolali",
"Kanwil BPN Provinsi Jawa Tengah",
#"Kabupaten Cilacap",
"DINAS PERHUBUNGAN",
#"Kabupaten Purworejo",
"DINAS SOSIAL",
#"Kabupaten Kebumen",
"DINAS PENDIDIKAN DAN KEBUDAYAAN",
#"Kabupaten Magelang",
#"Kabupaten Sukoharjo",
"DINAS PEKERJAAN UMUM SUMBER DAYA AIR DAN PENATAAN RUANG",
"DINAS LINGKUNGAN HIDUP DAN KEHUTANAN",
"DINAS PERUMAHAN RAKYAT DAN KAWASAN PERMUKIMAN",
#"Kabupaten Purbalingga",
#"Kabupaten Kudus",
"DINAS PERINDUSTRIAN DAN PERDAGANGAN",
#"Kabupaten Pekalongan",
"BIRO PEREKONOMIAN",
#"Kabupaten Sragen",
#"Kabupaten Karanganyar",
"BPJS Kesehatan",
#"Kabupaten Wonosobo",
#"Kabupaten Temanggung",
"BADAN PENGHUBUNG",
#"Kabupaten Rembang",
#"Kabupaten Wonogiri",
#"Kabupaten Batang",
##"DINAS KOPERASI DAN UMKM",
#"Kota Surakarta",
"DINAS PERTANIAN DAN PERKEBUNAN",
"BADAN PENANGGULANGAN BENCANA DAERAH",
#"Kabupaten Banjarnegara",
"Kanwil Kemenag Provinsi Jawa Tengah",
#"Kabupaten Tegal",
"BADAN KEPEGAWAIAN DAERAH",
##"INSPEKTORAT",
##"BIRO KESEJAHTERAAN RAKYAT",
#"Kota Tegal",
##"BANK JATENG",
##"DINAS KEPEMUDAAN, OLAHRAGA DAN PARIWISATA",
#"Kota Magelang",
#"Kota Salatiga",
##"BIRO UMUM",
#"Balai Besar Pelaksanaan Jalan Nasional Jawa Tengah & DIY",
##"DINAS KOMUNIKASI DAN INFORMATIKA",
##"DINAS PETERNAKAN DAN KESEHATAN HEWAN",
#"Kota Pekalongan",
"BPJS Ketenagakerjaan Kanwil Jateng dan DIY",
##"DINAS KELAUTAN DAN PERIKANAN",
##"DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SATU PINTU",
]
filtered_df = df[
    (df["Status"] == "Selesai") & (df["Departement"].isin(specific_departments))
]

# Keep only the 'Message', 'Status', and 'Departement' columns
filtered_df = filtered_df[["Message", "Status", "Departement"]]

# Save the preprocessed data to a new CSV file
filtered_df.to_csv("delapan juni filtered2.csv", index=False)

# Display the preprocessed DataFrame
print(len(specific_departments))
print(filtered_df)

22
                                                 Message   Status  \
0      Kami 120 orang guru yang sudah memiliki SK tun...  Selesai   
1      kenapa kab. brebes jalannya tidak pernah beres...  Selesai   
2      Pak Ganjar......untuk wilayah Blora Kota saat ...  Selesai   
3      Lapor Pak ... tolong birokrasi di Kecamatan ke...  Selesai   
4      Pupuk Urea di Kab. Semarang langka pak Gub...t...  Selesai   
...                                                  ...      ...   
65953  Izin melaporkan kegiatan galian liar, Dukuh pi...  Selesai   
65955  Selamat Pagi, Mohon perbaikan untuk jalan utam...  Selesai   
65959  Izin melaporkan galian liar di dukuh Pilangban...  Selesai   
65969  Lokasi aduan : ngembik lor jalan rambutan kram...  Selesai   
65984  Tolong survei jalan provinsi Slawi-Jatinegara ...  Selesai   

                                       Departement  
0                         BADAN KEPEGAWAIAN DAERAH  
1      DINAS PEKERJAAN UMUM BINA MARGA CIPTA KARYA  
2        

In [3]:
# Load the dataset
df = pd.read_csv("delapan juni filtered2.csv")


def preprocess_label(text):
    # Convert text to lowercase
    text = str(text).lower()
    return text


# Define a function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = str(text).lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r"[^a-z\s]", " ", text)

    # Tokenize the text
    tokens = text.split()

    # Remove stopwords
    stopword_factory = StopWordRemoverFactory()
    stopword_remover = stopword_factory.create_stop_word_remover()
    tokens = [stopword_remover.remove(word) for word in tokens]

    """# Stemming
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()
    tokens = [stemmer.stem(word) for word in tokens]"""

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)
    if preprocessed_text and preprocessed_text[0].isspace():
        preprocessed_text = preprocessed_text[1:]

    preprocessed_text = re.sub(r"\s+", " ", text)
    # print(preprocessed_text)
    return preprocessed_text

In [4]:
#3152
print(df['Message'][40])

jalan propinsi lasem-sale kab.rembang rusak parah pak 1 tahun ini, aspalnya sudah pada hilang,, tolong cepat dperbaiki.. karena itu wewenang provinsi bukan kabupaten.. kabupaten tidak mau memperbaiki karena itu jalan propinsi..


In [5]:
tqdm.pandas(desc="Preprocessing")
df["Message"] = df["Message"].progress_apply(preprocess_text)
df["Departement"] = df["Departement"].progress_apply(preprocess_label)


# Save the preprocessed dataset to a new CSV file
df.to_csv("delapan juni filtered2.csv", index=False)

Preprocessing: 100%|██████████| 26268/26268 [00:00<00:00, 222046.61it/s]


In [13]:
# Apply the preprocessing function to the text column
df['cleaned_text'] = df['Message'][:10].apply(preprocess_text)

# Save the preprocessed dataset to a new CSV file
df.to_csv('preprocessed_dataset.csv', index=False)

In [6]:
print(df['Message'][:10])

0    kami orang guru yang sudah memiliki sk tunjang...
1    kenapa kab brebes jalannya tidak pernah beres ...
2    pak ganjar untuk wilayah blora kota saat ini k...
3    lapor pak tolong birokrasi di kecamatan kecama...
4    pupuk urea di kab semarang langka pak gub tolo...
5    tolong pak gubernur di kecamatan sadang kab ke...
6    salam pak gubernur yang saya hormati mohon dic...
7    di desa tumbrep kecamatan bandar kabupaten bat...
8    lapor pak gubernur apa betul pencairan bansos ...
9    jalan rusak parah terlihat tak seperti jalan m...
Name: Message, dtype: object


In [ ]:
# Assuming df is your DataFrame containing the cleaned text
for cleaned_text in df['cleaned_text']:
    print(cleaned_text)
